<center><h1>chen_jacqueline_HW3</h1></center>
<br>
<br>

Name: Jacqueline Chen
<br>
Github Username: jchen741
<br>
USC ID: 7983593463

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [16]:
import pandas as pd
import os
path =os.getcwd()
print(path)
import numpy as np
from scipy.stats import bootstrap as boot_scip
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats


/Users/jacy/Documents/masters/dsci 552/homework/homework-3-jchen741/notebook


Get the AReM Data Set

In [17]:
filename = path + "/data/AReM/"
word = "/notebook"
# Replace the word and update the filename variable
filename = filename.replace(word, "")
# TESTING
# name = 'bending2'
# folder_path = filename + name+'/'
# test_list = 'dataset4.csv'
# file = folder_path+test_list
# df = pd.read_csv(file,skiprows = 5, delimiter = ' ', header = None)
# display(df)
def convertcsv(root_folder,test_list):
    # getting folders, like bending 1, bending 2, etc.
    folder_path=filename+root_folder+'/'
    # selected data sets from the input
    test_list=['dataset'+str(x)+'.csv' for x in test_list]
    # directory of all the files in the folder_path
    files=os.listdir(folder_path)
    # create list to store training and test data
    train_data = []
    test_data=[]
    for file in files:
        file_path=folder_path+file
        # print(file_path)
        dataset_number = int(file.split('dataset')[1].split('.')[0])
        # for dataset 4 in bending2 with space delimiter
        if (root_folder == 'bending2' and dataset_number in [4]):
            df = pd.read_csv(file_path,skiprows = 5, sep = '\s+', header = None)
            # process df_space as needed
        # for rewriting dataset to remove trailing comma on line 485
        elif (root_folder == 'cycling' and dataset_number in [9, 14]):
            with open(file_path, 'r') as file:
                lines = file.readlines()
                # identify and modify the last line
                last_line = lines[-1].rstrip(',')  # Remove trailing comma
                # write the modified content back to the file
            with open(file_path, 'w') as file:
                file.writelines(lines[:-1])  # Write all lines except the last
                file.write(last_line)
            df = pd.read_csv(file_path,skiprows = 5, header = None)
        # all other cases
        else:
            df = pd.read_csv(file_path,skiprows = 5, header = None)
        df.columns = ['time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
        # display(df)
        # df.dropna(inplace=True)
        # separating each dataset into training list and test lists
        if file in test_list:
            test_data.append(df)
        else:
            train_data.append(df)
    train_dict=[root_folder,train_data]
    test_dict=[root_folder,test_data]
    return train_dict,test_dict

### (b) Test and Train Data

In [18]:
def buildsets(train, test, data):
    train_dict=data[0]
    test_dict=data[1]
    train[train_dict[0]]=train_dict[1]
    #train.append(data[0])
    test[test_dict[0]]=test_dict[1]

In [19]:
train_data,test_data={},{}
buildsets(train_data, test_data, convertcsv('bending1',[1,2]))
buildsets(train_data, test_data, convertcsv('bending2',[1,2]))
buildsets(train_data, test_data, convertcsv('cycling',[1,2,3]))
buildsets(train_data, test_data, convertcsv('lying',[1,2,3]))
buildsets(train_data, test_data, convertcsv('sitting',[1,2,3]))
buildsets(train_data, test_data, convertcsv('walking',[1,2,3]))
buildsets(train_data, test_data, convertcsv('standing',[1,2,3]))

### (c) Feature Extraction

#### i. Research

Minimum: The smallest value in the time series.
</br>
Maximum: The largest value in the time series.
</br>
Mean: The average value of the time series data points.
</br>
Median: The middle value of the sorted time series data.
</br>
Standard Deviation: A measure of the spread or dispersion of the data points around the mean.
</br>
Variance: The average of the squared differences from the Mean.
</br>
Skewness: A measure of the asymmetry of the probability distribution of the time series.
</br>
Percentiles (e.g., 25th, 50th, 75th): Values below which a given percentage of observations fall.
</br>
Interquartile Range (IQR): The range between the 75th percentile and 25th percentile.
</br>
Entropy: A measure of the disorder or unpredictability in the time series.

#### ii. Extraction

In [20]:
# since the data was separated into training and test sets,
# we should compute them separately first then bring them together
def new_columns(value,label,train_or_test):
    #display(value)
    old_columns=value.columns[1:]
    # extractin time domain features
    min=value.min()
    max=value.max()
    mean=value.mean()
    std=value.std()
    median=value.median()
    first_quartile=value.quantile(0.25)
    third_quartile=value.quantile(0.75)
    row=[]
    # appending all values into one column for each instance
    for column in old_columns:
        row.append(min[column])
        row.append(max[column])
        row.append(mean[column])
        row.append(std[column])
        row.append(median[column])
        row.append(first_quartile[column])
        row.append(third_quartile[column])
    row.append(label)
    row.append(train_or_test)
    return row
    #print(row)

In [21]:
# creating a function to produce subscripts
def subscript_number(number):
    subscript_map = {'0': '\u2080', '1': '\u2081', '2': '\u2082', '3': '\u2083', '4': '\u2084', '5': '\u2085', '6': '\u2086', '7': '\u2087', '8': '\u2088', '9': '\u2089'}
    return ''.join([subscript_map[digit] for digit in number])
def make_new_dataset(data,train_or_test):
    col1=['1','2','3','4','5','6']
    col2=['min','max','mean','std','median','1st quart','3rd quart']
    new_col=[]
    # column names
    for right in col1:
        for left in col2:
            new_col.append(left+subscript_number(right))
    new_col.append('class')
    new_col.append('train_or_test')
    new_data=[]
    for key in data:
        for value in data[key]:
            #display(value)
            new_data.append(new_columns(value,key,train_or_test))
    #print(len(data_new))
    return pd.DataFrame(new_data,columns=new_col)

In [22]:
train_df=make_new_dataset(train_data,'train')
train_df

,min₁,max₁,mean₁,std₁,median₁,1st quart₁,3rd quart₁,min₂,max₂,mean₂,...,3rd quart₅,min₆,max₆,mean₆,std₆,median₆,1st quart₆,3rd quart₆,class,train_or_test
0,36.25,48.00,43.969125,1.618364,44.50,43.3100,44.67,0.0,1.50,0.413125,...,23.75,0.0,2.96,0.555312,0.487826,0.49,0.00,0.83,bending1,train
1,37.00,48.00,43.454958,1.386098,43.25,42.5000,45.00,0.0,1.58,0.378083,...,24.00,0.0,5.26,0.679646,0.622534,0.50,0.43,0.87,bending1,train
2,33.00,47.75,42.179812,3.670666,43.50,39.1500,45.00,0.0,3.00,0.696042,...,36.33,0.0,2.18,0.613521,0.524317,0.50,0.00,1.00,bending1,train
3,33.00,45.75,41.678063,2.243490,41.75,41.3300,42.75,0.0,2.83,0.535979,...,31.25,0.0,1.79,0.383292,0.389164,0.43,0.00,0.50,bending1,train
4,35.00,47.40,43.954500,1.558835,44.33,43.0000,45.00,0.0,1.70,0.426250,...,36.50,0.0,1.79,0.493292,0.513506,0.43,0.00,0.94,bending1,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,36.25,45.50,42.959354,1.500878,42.67,42.0000,44.33,0.0,2.60,0.352875,...,16.69,0.0,4.00,0.748479,0.461152,0.82,0.43,0.95,standing,train
65,36.00,47.33,44.441187,2.417797,45.00,44.6275,45.75,0.0,4.50,0.346604,...,14.67,0.0,5.91,1.155083,0.842087,0.94,0.50,1.50,standing,train
66,36.25,45.75,43.187521,2.491162,44.75,39.7500,45.00,0.0,2.83,0.271271,...,21.00,0.0,3.83,0.645458,0.567419,0.50,0.43,0.83,standing,train
67,29.75,48.00,46.932208,1.832665,47.50,47.2375,47.75,0.0,4.60,0.429667,...,15.50,0.0,6.58,0.991125,0.855329,0.83,0.47,1.22,standing,train


In [23]:
test_df = make_new_dataset(test_data, 'test')
test_df

,min₁,max₁,mean₁,std₁,median₁,1st quart₁,3rd quart₁,min₂,max₂,mean₂,...,3rd quart₅,min₆,max₆,mean₆,std₆,median₆,1st quart₆,3rd quart₆,class,train_or_test
0,37.25,45.00,40.624792,1.476967,40.500,39.2500,42.0000,0.0,1.30,0.358604,...,36.0000,0.0,1.92,0.570583,0.582915,0.430,0.0000,1.300,bending1,test
1,38.00,45.67,42.812812,1.435550,42.500,42.0000,43.6700,0.0,1.22,0.372437,...,34.5000,0.0,3.11,0.571083,0.601010,0.430,0.0000,1.300,bending1,test
2,12.75,51.00,24.562958,3.737514,24.250,23.1875,26.5000,0.0,6.87,0.590833,...,27.0000,0.0,4.97,0.700188,0.693720,0.500,0.4300,0.870,bending2,test
3,0.00,42.75,27.464604,3.583582,28.000,25.5000,30.0000,0.0,7.76,0.449708,...,20.7500,0.0,6.76,1.122125,1.012342,0.830,0.4700,1.300,bending2,test
4,24.25,45.00,37.177042,3.581301,36.250,34.5000,40.2500,0.0,8.58,2.374208,...,21.7500,0.0,9.34,2.921729,1.852600,2.500,1.5000,3.900,cycling,test
5,28.75,44.75,37.561187,3.226507,36.875,35.2500,40.2500,0.0,9.91,2.080687,...,21.5000,0.0,9.62,2.765896,1.769203,2.450,1.4100,3.770,cycling,test
6,22.00,44.67,37.058708,3.710180,36.000,34.5000,40.0625,0.0,14.17,2.438146,...,21.0000,0.0,8.55,2.983750,1.815730,2.570,1.5000,4.150,cycling,test
7,23.50,30.00,27.716375,1.442253,27.500,27.0000,29.0000,0.0,1.79,0.363687,...,10.7500,0.0,4.50,0.734271,0.613688,0.710,0.4300,1.000,lying,test
8,24.75,48.33,44.182937,7.495615,48.000,48.0000,48.0000,0.0,3.11,0.101875,...,5.5425,0.0,3.91,0.692771,0.675781,0.500,0.3225,0.940,lying,test
9,48.00,48.25,48.004167,0.032038,48.000,48.0000,48.0000,0.0,0.43,0.007167,...,10.0000,0.0,2.50,0.641229,0.388372,0.500,0.4600,0.830,lying,test


In [24]:
#combining both training and test data sets
full_df=pd.concat([train_df,test_df],ignore_index=True)
full_df

,min₁,max₁,mean₁,std₁,median₁,1st quart₁,3rd quart₁,min₂,max₂,mean₂,...,3rd quart₅,min₆,max₆,mean₆,std₆,median₆,1st quart₆,3rd quart₆,class,train_or_test
0,36.25,48.00,43.969125,1.618364,44.500,43.31,44.6700,0.0,1.50,0.413125,...,23.750,0.0,2.96,0.555312,0.487826,0.490,0.00,0.83,bending1,train
1,37.00,48.00,43.454958,1.386098,43.250,42.50,45.0000,0.0,1.58,0.378083,...,24.000,0.0,5.26,0.679646,0.622534,0.500,0.43,0.87,bending1,train
2,33.00,47.75,42.179812,3.670666,43.500,39.15,45.0000,0.0,3.00,0.696042,...,36.330,0.0,2.18,0.613521,0.524317,0.500,0.00,1.00,bending1,train
3,33.00,45.75,41.678063,2.243490,41.750,41.33,42.7500,0.0,2.83,0.535979,...,31.250,0.0,1.79,0.383292,0.389164,0.430,0.00,0.50,bending1,train
4,35.00,47.40,43.954500,1.558835,44.330,43.00,45.0000,0.0,1.70,0.426250,...,36.500,0.0,1.79,0.493292,0.513506,0.430,0.00,0.94,bending1,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,12.50,45.00,33.509729,4.850923,34.125,30.50,36.7500,0.0,13.05,4.450771,...,18.750,0.0,8.96,3.378479,1.787360,3.085,2.06,4.44,walking,test
84,15.00,46.75,34.660583,5.315110,35.000,31.00,38.2500,0.0,13.44,4.200896,...,18.500,0.0,8.99,3.244396,1.630983,3.000,2.12,4.24,walking,test
85,33.33,48.00,44.334729,2.476940,45.000,42.25,46.5000,0.0,3.90,0.432958,...,17.750,0.0,5.02,0.933000,0.673609,0.830,0.47,1.25,standing,test
86,35.50,46.25,43.174938,1.989052,43.670,42.50,44.5000,0.0,2.12,0.506583,...,16.500,0.0,5.72,0.911979,0.666161,0.830,0.47,1.22,standing,test


#### iii. Standard Deviation

In [25]:
for column in full_df.columns[:-2]:
    std_estimates = full_df[column].std()
    print(column,':',std_estimates)

min₁ : 9.569975284512807
max₁ : 4.394361684703191
mean₁ : 5.3357176583441746
std₁ : 1.7721533975028414
median₁ : 5.440054137152636
1st quart₁ : 6.1535897241853705
3rd quart₁ : 5.138924613450999
min₂ : 0.0
max₂ : 5.0627293748106394
mean₂ : 1.574163920074514
std₂ : 0.8841054781862712
median₂ : 1.4122441649745636
1st quart₂ : 0.946386191125693
3rd quart₂ : 2.1252664064059106
min₃ : 2.956462059205818
max₃ : 4.875136693954745
mean₃ : 4.008380131777648
std₃ : 0.9467102620752953
median₃ : 4.036396318784075
1st quart₃ : 4.22065788450763
3rd quart₃ : 4.171628224016796
min₄ : 0.0
max₄ : 2.183625484972341
mean₄ : 1.166114090919611
std₄ : 0.45824169747631144
median₄ : 1.145585660988056
1st quart₄ : 0.8436196840035467
3rd quart₄ : 1.5525041782503672
min₅ : 6.124001430553486
max₅ : 5.741238300951952
mean₅ : 5.675593204313176
std₅ : 1.0248979053506881
median₅ : 5.813782436530036
1st quart₅ : 6.096465201564951
3rd quart₅ : 5.531720219299494
min₆ : 0.04583815401645622
max₆ : 2.5189209776905526
mean₆ : 

In [26]:
from bootstrapped.stats_functions import std, mean

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Feature'])
result_list =[]
for feature in full_df.columns[:-2]:
    # Calculate bootstrap confidence interval for standard deviation
    result = bs.bootstrap(full_df[feature].values, stat_func=std, alpha=0.10, num_iterations=10000)
    standard_dev = result.value
    ci_lower = result.lower_bound
    ci_upper = result.upper_bound
    result_list.append({'Feature':feature, 'Results':standard_dev,'CI Lower': ci_lower, 'CI Upper': ci_upper})
results_df = pd.DataFrame(result_list)
print(results_df)

       Feature   Results  CI Lower   CI Upper
0         min₁  9.515445  8.308500  10.821870
1         max₁  4.369322  3.490869   5.425734
2        mean₁  5.305314  4.750977   5.941340
3         std₁  1.762056  1.584269   1.961663
4      median₁  5.409056  4.847812   6.047855
5   1st quart₁  6.118526  5.631529   6.707453
6   3rd quart₁  5.109643  4.399071   5.914707
7         min₂  0.000000  0.000000   0.000000
8         max₂  5.033882  4.693506   5.467651
9        mean₂  1.565194  1.434757   1.742488
10        std₂  0.879068  0.823332   0.960430
11     median₂  1.404197  1.274161   1.578655
12  1st quart₂  0.940994  0.852691   1.056571
13  3rd quart₂  2.113157  1.943795   2.348211
14        min₃  2.939616  2.789547   3.134451
15        max₃  4.847358  4.265093   5.555346
16       mean₃  3.985540  3.503668   4.567937
17        std₃  0.941316  0.763667   1.123473
18     median₃  4.013397  3.525075   4.611855
19  1st quart₃  4.196608  3.714985   4.790646
20  3rd quart₃  4.147858  3.636970

In [27]:
# from scipy
scipy_list = []
for column in full_df.columns[:-2]:
    result_scipy = boot_scip((full_df[column],), np.std, confidence_level = 0.90)
    ci_l, ci_u = result_scipy.confidence_interval
    scipy_list.append({'Feature':column,'CI Lower': ci_l, 'CI Upper': ci_u})
scipy_boot=pd.DataFrame(scipy_list)
scipy_boot.fillna(0, inplace=True)
print(scipy_boot)

/Users/jacy/anaconda3/lib/python3.11/site-packages/scipy/stats/_resampling.py:144: RuntimeWarning: invalid value encountered in scalar divide
  a_hat = 1/6 * sum(nums) / sum(dens)**(3/2)
/Users/jacy/anaconda3/lib/python3.11/site-packages/scipy/stats/_resampling.py:97: DegenerateDataWarning: The BCa confidence interval cannot be calculated. This problem is known to occur when the distribution is degenerate or the statistic is np.min.
  warnings.warn(DegenerateDataWarning(msg))


       Feature  CI Lower   CI Upper
0         min₁  8.501490  11.143622
1         max₁  3.517626   5.497131
2        mean₁  4.793369   5.982942
3         std₁  1.600097   1.988578
4      median₁  4.902910   6.147966
5   1st quart₁  5.636421   6.714031
6   3rd quart₁  4.456117   5.988987
7         min₂  0.000000   0.000000
8         max₂  4.682167   5.459056
9        mean₂  1.407997   1.712018
10        std₂  0.810007   0.943491
11     median₂  1.249074   1.545807
12  1st quart₂  0.840565   1.039383
13  3rd quart₂  1.907465   2.301521
14        min₃  2.780077   3.126803
15        max₃  4.250277   5.526732
16       mean₃  3.490841   4.556731
17        std₃  0.804275   1.208362
18     median₃  3.513919   4.611646
19  1st quart₃  3.676693   4.757795
20  3rd quart₃  3.612886   4.747592
21        min₄  0.000000   0.000000
22        max₄  2.004187   2.391533
23       mean₄  1.074974   1.217253
24        std₄  0.424954   0.488005
25     median₄  1.054384   1.197234
26  1st quart₄  0.778017   0

#### iv. Select Features

Just to summarize what this project tells me:
<br>
Building a 90% bootstrap confidence interval for the standard deviation for each of the six time series features: <br>avg rss12, var rss12, avg rss13, var rss13, vg rss23, and ar rss23
<br>
essentially shows that 90% of the time, the feature's min, max, etc. ranges from the ci_lower to ci_upper.
<br>
Some features have zero variability, like min_2 and min_4 and that means it does not add any variability, or information, to the model. <br>

Therefore, I would not select min as an important feature.<br>

The more variability the data has, more information it brings.
So when selecting important features, I want good variability. <br>

For outliers, be careful with features that are prone to outlier, which would be the max features because it would always get the outliers.<br>

All in all, the three most important time-domain features I would select would be mean, median, and standard deviation because it would help in distinguishing different classes.


## 2. ISLR 3.7.4

### (a) Linear Train

RSS is the residual sum of squares. If RSS is lower, it means that the model fits the training data better and the model's predictions are closer to the actual observed values. If the model's true relationship is linear, training for the cubic regression would be unnecessary and project a higher RSS than the RSS for the linear regression. The cubic equation would extremely overfit the data as well. As a result, the train RSS will keep reducing as the degree of polynomial increases.

### (b) Linear Test

Because the cubic regression from training is already overfit, it would perform even worse on the test set compared to the test set of the linear regression model. As a result, the linear regression would still have a lower test RSS than the RSS of the cubic test model.

### (c) Not Linear Train

Without more information, determining whether one RSS is lower than the other is difficult to determine. We would need to know how far the true relationship varies between linearity and cubic regression. But, generally I would think that increasing the degree of the polynomial would lower the training RSS for the data.

### (d) Not Linear Testing

For the test RSS of linear and cubic regression, there is not enough information becasue it would truely depend completely on the true relationship. It would possibly require model selection, like cross validation, to determine which model fits better. If the model leans more towards the cubic relationship, the RSS for test would be lower than the linear RSS.